In [1]:
import sys
sys.path.append('../utils')
import fvdb
import fvdb_utils
import mesh_tools as mt
import igl
import numpy as np
from ssfid import calculate_activation_statistics, calculate_frechet_distance
from patch_utils import pairwise_IoU_dist
from classifier3D import classifier
import torch
import os
import glob
from tqdm import tqdm
import trimesh 

device='cuda'

In [2]:
voxel_size = 256
model = classifier(voxel_size=voxel_size)

weights_path = 'Clsshapenet_'+str(voxel_size)+'.pth'
if not os.path.exists(weights_path):
    raise RuntimeError(
        f"'{weights_path}' not exists. Please download it from https://drive.google.com/file/d/1HjnDudrXsNY4CYhIGhH4Q0r3-NBnBaiC/view?usp=sharing.")
model.load_state_dict(torch.load(weights_path, weights_only=False))
model.to(device)
model.eval()

names = os.listdir("../../data/GT_WT")
names.sort()

In [3]:
def sample_grid_points_aabb(aabb, resolution):
    aabb_min, aabb_max = aabb[:3], aabb[3:]
    aabb_size = aabb_max - aabb_min
    resolutions = (resolution * aabb_size / aabb_size.max()).astype(np.int32)

    xs = np.linspace(0.5, resolutions[0] - 0.5, resolutions[0]) / resolutions[0] * aabb_size[0] + aabb_min[0]
    ys = np.linspace(0.5, resolutions[1] - 0.5, resolutions[1]) / resolutions[1] * aabb_size[1] + aabb_min[1]
    zs = np.linspace(0.5, resolutions[2] - 0.5, resolutions[2]) / resolutions[2] * aabb_size[2] + aabb_min[2]
    grid_points = np.stack(np.meshgrid(xs, ys, zs, indexing='ij'), axis=-1)
    return grid_points

def normalize_aabb(v, reso, enlarge_scale=1.03, mult=8):
    aabb_min = np.min(v, axis=0)
    aabb_max = np.max(v, axis=0)
    center = (aabb_max + aabb_min) / 2
    bbox_size = (aabb_max - aabb_min).max() * enlarge_scale
    translation = -center
    scale = 1.0 / bbox_size * 2
    # v = (v + translation) * scale
    # v = (v - center) / bbox_size * 2
    aabb_min = (aabb_min * enlarge_scale - center) / bbox_size * 2
    aabb_max = (aabb_max * enlarge_scale - center) / bbox_size * 2
    aabb = np.concatenate([aabb_min, aabb_max], axis=0)
    aabb_size = aabb_max - aabb_min
    fm_size = (reso * aabb_size / aabb_size.max()).astype(np.int32)
    # round to multiple of 8
    fm_size = (fm_size + mult - 1) // mult * mult
    aabb_max = fm_size / fm_size.max()
    aabb = np.concatenate([-aabb_max, aabb_max], axis=0)
    return aabb, translation, scale

In [4]:
def get_dense_array_tight(shape_path, pts=None, translation=None, scale=None):
    ms = trimesh.load(shape_path)
    v = ms.vertices
    f = ms.faces
    # v, f = igl.read_triangle_mesh(shape_path)
    if pts is None:
        out_grid = True
        v = 2*mt.NDCnormalize(v)
        aabb, translation, scale = normalize_aabb(v, 256)
        v = (v+translation)*scale
        pts = sample_grid_points_aabb(aabb, 256)

    else:
        out_grid = False
        v = (v+translation)*scale
        
    sdf_compute = igl.fast_winding_number_for_meshes(v, f, pts.reshape(-1, 3))>.5
    or_sdfgrid = sdf_compute.reshape(pts.shape[:-1])
    new_shape = [int(x * voxel_size / max(or_sdfgrid.shape))
                        for x in or_sdfgrid.shape]

    or_sdfgrid = torch.nn.functional.adaptive_max_pool3d(1.*torch.tensor(or_sdfgrid[None, None], device=device), new_shape)[0, 0]>0
    or_sdfgrid = 1.*or_sdfgrid
    if out_grid:
        return or_sdfgrid, pts, translation, scale
    return or_sdfgrid


In [5]:

def get_metrics(globe, mesh_name):
    shapes = glob.glob(globe.format(mesh_name))
    shapes.sort()
    # print(shapes)
    or_sdfgrid, pts, translation, scale = get_dense_array_tight("../../data/GT_WT/{}.obj".format(mesh_name))
    mu_r, sigma_r = calculate_activation_statistics(or_sdfgrid, model)
    ssfid_values = []
    grids = []
    for shape in shapes:
        dense_grid = get_dense_array_tight(shape, pts, translation, scale)
        mu_f, sigma_f = calculate_activation_statistics(dense_grid, model)
        ssfid = calculate_frechet_distance(mu_r, sigma_r, mu_f, sigma_f)
        print(ssfid)
        ssfid_values.append(ssfid)
        grids.append(dense_grid)
    return np.mean(ssfid_values).round(4), pairwise_IoU_dist(torch.stack(grids, dim=0)).round(4)



### Ours

In [8]:
names = [e[:-4] for e in os.listdir('../../data/GT_WT')]
names.sort()
str_path = '../../textured_output/{}/**/large_mesh.ply'

ours_ssfid = []
ours_iou = []
for name in tqdm(names):
    ssfid, io = get_metrics(str_path, name)
    ours_ssfid.append(ssfid)
    ours_iou.append(io)
  

  0%|          | 0/8 [00:00<?, ?it/s]

0.004732269651512411
0.00307772070581791
0.007658762343396575
0.005493369046789098
0.0062049652391635846
0.004415247648871912
0.004356549786081132
0.007717780726949286
0.004497226279355004


 12%|█▎        | 1/8 [00:31<03:39, 31.37s/it]

0.007479221466766717
0.02479684906095514
0.06856322977947116
0.04666025168242527
0.016465378277686682
0.13155625727571874
0.28342016796712954
0.13468316837840177
0.039199289514925795
0.3286345817660248


 25%|██▌       | 2/8 [00:47<02:15, 22.53s/it]

...

0.33169628840485643
0.06617421377750787
0.16362994264123643
0.09559280376041812
0.06285362764987212
0.30394454837421847
0.056022154544919545
0.011202007935864344
0.20972336967756178
0.06153999496918061


 88%|████████▊ | 7/8 [02:13<00:17, 17.63s/it]

0.0502383576527663
0.0007669216777372867
0.003154429412042248
0.052025607269349905
0.04314092928968449
0.018508777197766335
0.008168825054688966
0.000799816224983374
0.0008762311451349092
0.0009487346467835778


100%|██████████| 8/8 [02:31<00:00, 18.98s/it]

0.0407359373369971


In [9]:
np.array(ours_ssfid).mean()

0.0970875

In [10]:
np.array(ours_iou).mean()

0.19713749999999997

In [11]:
ssfid_data = ours_ssfid + [np.array(ours_ssfid).mean()]
iou_data = ours_iou + [np.array(ours_iou).mean()]

In [12]:
ssfid_data = ours_ssfid + [np.array(ours_ssfid).mean()]
iou_data = ours_iou + [np.array(ours_iou).mean()]
print(r"\begin{tabular}{ l |c| " +"".join([' c ' for e in names]) + r" | c }")

print(r'Metric & Method               & {} & mean \\'.format(' & '.join(names)))
print(r'\hline')
print(r'G-Qual. $\downarrow$ & ours & {}\\'.format('   &    '.join(['{:.3f}' for e in ssfid_data]).format(*ssfid_data)))
print(r'\hline') 
print(r'G-Div. $\uparrow$ & ours    & {}\\'.format('   &    '.join(['{:.3f}' for e in iou_data]).format(*iou_data)))
print(r"\end{tabular}")


\begin{tabular}{ l |c|  c  c  c  c  c  c  c  c  | c }
Metric & Method               & acropolis & canyon & fighting-pillar & house & ruined-tower & small-town & stone-cliff & wood & mean \\
\hline
G-Qual. $\downarrow$ & ours & 0.006   &    0.111   &    0.080   &    0.005   &    0.071   &    0.379   &    0.108   &    0.017   &    0.097\\
\hline
G-Div. $\uparrow$ & ours    & 0.026   &    0.210   &    0.100   &    0.008   &    0.401   &    0.536   &    0.208   &    0.089   &    0.197\\
\end{tabular}


In [21]:

# print(r"\begin{tabular}{ l |c| " +"".join([' c ' for e in names]) + r"}")

# print(r'Metric & Method               & {} \\'.format(' & '.join(names)))
# print(r'\hline')
# print(r'G-Qual. $\downarrow$ & ours & {} \\'.format('   &    '.join(['{:.3f}' for e in ours_ssfid]).format(*ours_ssfid)))
# print(r'\hline') 
# print(r'G-Div. $\uparrow$ & ours    & {} \\'.format('   &    '.join(['{:.3f}' for e in ours_iou]).format(*ours_iou)))
# print(r"\end{tabular}")


### Sin3DM vs ground truth

In [ ]:
names = os.listdir("../data/gen_SIN3DM_normalize")
names.sort()

'Metric & Method & ' +' & '.join(names)
Sin3DM_ssfid = []
Sin3DM_iou = []
for name in tqdm(names):
    ssfid, io = get_metrics('../data/gen_SIN3DM_normalize/{}/**.obj', name)
    Sin3DM_ssfid.append(ssfid)
    Sin3DM_iou.append(io)
  

In [ ]:

print(r'G-Qual. $\downarrow$ & Sin3DM (v. GT, ${}^3$) & {} \\'.format(voxel_size, '   &    '.join(['{:.3f}' for e in Sin3DM_ssfid]).format(*Sin3DM_ssfid)))
print(r'\hline') 
print(r'G-Div. $\uparrow$ & Sin3DM (v. GT, ${}^3$)   & {} \\'.format(voxel_size, '   &    '.join(['{:.3f}' for e in Sin3DM_iou]).format(*Sin3DM_iou)))


In [ ]:

print(r'G-Qual. $\downarrow$ & Sin3DM (v. GT, ${}^3$) & {} \\'.format(voxel_size, '   &    '.join(['{:.3f}' for e in Sin3DM_ssfid]).format(*Sin3DM_ssfid)))
print(r'\hline') 
print(r'G-Div. $\uparrow$ & Sin3DM (v. GT, ${}^3$)   & {} \\'.format(voxel_size, '   &    '.join(['{:.3f}' for e in Sin3DM_iou]).format(*Sin3DM_iou)))


### SIN3DGen

In [ ]:
name='acropolis'
shapes = glob.glob('../data/gen_Sin3DGen/{}/generated/*/mesh_normalized2.obj'.format(name))
shapes.sort()
# print(shapes)
or_sdfgrid, pts, translation, scale = get_dense_array_tight("../data/SIN3DM_WT/{}.obj".format(name))
for shape in shapes:
    dense_grid = get_dense_array_tight(shape, pts, translation, scale)
    break

In [ ]:
mt.plotSlice(dense_grid.cpu().detach().numpy(), 1.)

In [ ]:
mt.plotSlice(or_sdfgrid.cpu().detach().numpy(), 1.)

In [ ]:
names = os.listdir("../data/gen_SIN3DM_normalize")
names.sort()

'Metric & Method & ' +' & '.join(names)
Sin3DM_ssfid = []
Sin3DM_iou = []
for name in tqdm(names):
    ssfid, io = get_metrics('../data/gen_Sin3DGen/{}/generated/*/mesh_normalized2.obj', name)
    Sin3DM_ssfid.append(ssfid)
    Sin3DM_iou.append(io)
  

In [ ]:
print(r'Metric & Method               & acropolis & canyon & fighting-pillar & house & ruined-tower & small-town & stone-cliff & wood & mean \\')

In [ ]:

print(r'G-Qual. $\downarrow$ & Sin3DGen & {} \\'.format('   &    '.join(['{:.3f}' for e in Sin3DM_ssfid]).format(*Sin3DM_ssfid)))
print(r'\hline') 
print(r'G-Div. $\uparrow$ & Sin3DGen & {} \\'.format('   &    '.join(['{:.3f}' for e in Sin3DM_iou]).format(*Sin3DM_iou)))


In [ ]:

print(r'G-Qual. $\downarrow$ & Sin3DGen & {} \\'.format('   &    '.join(['{:.3f}' for e in Sin3DM_ssfid]).format(*Sin3DM_ssfid)))
print(r'\hline') 
print(r'G-Div. $\uparrow$ & Sin3DGen & {} \\'.format('   &    '.join(['{:.3f}' for e in Sin3DM_iou]).format(*Sin3DM_iou)))


In [ ]:
'   &    '.join(['{:.3f}' for e in Sin3DM_ssfid]).format(*Sin3DM_ssfid)

### SingleShapeGen

In [19]:
names = os.listdir("../data/gen_SIN3DM_normalize")
names.sort()

'Metric & Method & ' +' & '.join(names)
Sin3DM_ssfid = []
Sin3DM_iou = []
ssfid, io = get_metrics('/home/nmaruani/ReImpl/SingleShapeGen/eval/{}/*.obj', 'house')
print(ssfid, io)


0.9112766021164163
0.917415638895676
0.9059105065902742
0.909388446260607
0.9138022859380612
0.9088840802786251
0.9003095853019971
0.9121874921727766
0.9124322647566316
0.9029681114707415
0.9095 0.0122


### Make mean

In [ ]:
import numpy as np

L1 = """Sin3DGen & 4.832   &    6.158   &    8.453   &    17.951   &    6.984   &    4.020   &    13.023   &    10.323
Sin3DM & 0.915   &    2.227   &    0.262   &    2.009   &    0.495   &    0.845   &    0.023   &    0.086 
ours & 0.007   &    0.211   &    0.263   &    0.006   &    0.109   &    1.005   &    0.099   &    0.020
Sin3DGen & 0.252   &    0.498   &    0.594   &    0.406   &    0.864   &    0.702   &    0.646   &    0.438
Sin3DM  & 0.123   &    0.173   &    0.150   &    0.011   &    0.208   &    0.600   &    0.316   &    0.102 
ours & 0.039   &    0.188   &    0.237   &    0.007   &    0.315   &    0.598   &    0.230   &    0.081"""

m = L1.replace(" ", "").split('\n')
names = [e.split('&')[0] for e in m]
res = np.array([e.split('&')[1:] for e in m]).astype(float)
means = res.mean(1)
for i in range(6):
    print(names[i] +'  &  ' + '   &    '.join(['{:.2f}' for e in res[i]]).format(*res[i]) + f' &  {means[i]:.2f}')

### Noise

In [34]:
m = torch.rand(2,256,256,256)>.5

In [39]:
m = torch.zeros((2,256,256,256))
for i in range(len(m)):
    m[i,i]=1

In [40]:
pairwise_IoU_dist(m)

1.0